In [3]:
--- Libraries:
arguments = require 'Settings.arguments'
constants = require 'Settings.constants'
card_to_string = require 'Game.card_to_string_conversion'
card_tools = require 'Game.card_tools'
game_settings = require 'Settings.game_settings'
Plot = require 'itorch.Plot'
nninit = require 'nninit'
CFRDGadget = torch.class('CFRDGadget')

require 'torch'
require 'math'
require 'Tree.tree_builder'
require 'Tree.tree_data_generation'
require 'Tree.tree_values'
require 'Tree.tree_visualiser'
require 'nn'
require 'Tree.tree_cfr'
require 'Tree.tree_cfr2'
require 'nngraph'
require 'optim'
require 'image'
require 'NN.strategy_net_builder'
require 'NN.strategy_net_trainer'
require 'Tree.tree_warm_start'
require 'Nn.StrategyCriterion'




In [4]:
--- Create the tree
builder = PokerTreeBuilder()
--- Parameters for the tree
params = {}
params.root_node = {}
params.root_node.board = card_to_string:string_to_board('As')
params.root_node.street = 2
params.root_node.current_player = constants.players.P1
params.root_node.bets = arguments.Tensor{200, 200}
--- CFR Solver
starting_ranges = arguments.Tensor(constants.players_count, game_settings.card_count)
starting_ranges[1]:copy(card_tools:get_uniform_range(params.root_node.board))
starting_ranges[2]:copy(card_tools:get_uniform_range(params.root_node.board))

In [ ]:
cfrd = CFRDGadget(params.root_node.board, starting_ranges[1], opponent_cfvs)

In [3]:
--- Get Data from 2000 iterations
number_of_iterations = 200
train_tree = builder:build_tree(params)
data = TreeData(train_tree)
train_tree_cfr = TreeCFR()
train_tree_cfr:run_cfr(train_tree, starting_ranges,number_of_iterations)
data:get_training_set(train_tree,1)

### Training the Model

In [4]:
-- Building the neural net model
nn_builder = StrategyNN()
nn_model = nn_builder.model
-- Building trainer:
nn_trainer = NNTrainer(data,nn_model)
-- Options:
opt = {}
opt.learningRate = 0.001
opt.momentum = 0.95
opt.nesterov = true
opt.validate = true

In [5]:
current_criterion = nn.SmoothL1Criterion()
train_lossL1,test_lossL1 = nn_trainer:train(nn_model,current_criterion,opt,1000)

In [6]:
local num_iter = torch.range(1,train_lossL1:size(1))
local plot = Plot()

plot:line(num_iter,train_lossL1,'black' ,'No Warm start CFR = Uniform warm start')
plot:line(num_iter,test_lossL1,'red' ,'No Warm start CFR = Uniform warm start')
plot:xaxis('Number of iterations'):yaxis('Loss'):redraw()

In [7]:
pred_strat = nn_model:forward({nn_trainer.all_data.features,nn_trainer.all_data.masks})
true_strat =nn_trainer.all_data.targets
print(torch.mean(torch.abs(pred_strat- true_strat)))

0.22011543701713


In [8]:
-- Building the neural net model
nn_builder = StrategyNN()
nn_model = nn_builder.model
-- Building trainer:
nn_trainer = NNTrainer(data,nn_model)
-- Options:
opt = {}
opt.learningRate = 0.001
opt.momentum = 0.85
opt.validate = true
opt.weightDecay = 0.75
---opt.nesterov = true

In [9]:
strategy_criterion = StrategyLoss()
train_loss,test_loss = nn_trainer:train(nn_model,strategy_criterion,opt,1000)

In [10]:
pred_strat = nn_model:forward({nn_trainer.all_data.features,nn_trainer.all_data.masks})
true_strat = nn_trainer.all_data.targets
print(torch.mean(torch.abs(pred_strat- true_strat)))

0.025086532490288


In [ ]:
train_matrix = torch.FloatTensor()
test_matrix = torch.FloatTensor()
number_iter = 620

for i = 1,50 do
    -- Building the neural net model
    nn_builder = StrategyNN()
    nn_model = nn_builder.model
    -- Building trainer:
    nn_trainer = NNTrainer(data,nn_model)
    -- Options:
    opt = {}
    opt.learningRate = 0.001
    opt.momentum = 0.85
    opt.validate = true
    opt.weightDecay = 0.75
    ---opt.nesterov = true
    strategy_criterion = StrategyLoss()
    ---strategy_criterion = nn.SmoothL1Criterion()
    train_loss,test_loss = nn_trainer:train(nn_model,strategy_criterion,opt,number_iter)
    train_matrix = train_matrix:cat(train_loss:view(1,number_iter),1)
    test_matrix = test_matrix:cat(test_loss:view(1,number_iter),1)
end

In [ ]:
local num_iter = torch.range(1,train_loss:size(1))
local plot = Plot()

plot:line(num_iter,train_matrix[1],'blue')
plot:line(num_iter,test_matrix[1],'red')
plot:legend(true):title('Train error vs Test error')
plot:xaxis('Number of iterations'):yaxis('Loss')
for i =2,50 do
    plot:line(num_iter,train_matrix[i],'blue')
    plot:line(num_iter,test_matrix[i],'red')
    
end
plot:draw()

    